In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions

%matplotlib inline

2025-11-30 16:33:52.779114: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-30 16:33:53.066466: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-30 16:34:12.586924: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-30 16:34:25.150552: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [2]:
#!git clone https://github.com/alexeygrigorev/clothing-dataset-small.git

In [3]:
#!pip install tensorflow

In [4]:
path = './clothing-dataset-small/train/t-shirt/'
name = '5f0a3fa0-6a3d-4b68-b213-72766a643de7.jpg'
full_name = f'{path}/{name}'

In [5]:
img = load_img(full_name, target_size=(299, 299))

In [6]:
print(img)

<PIL.Image.Image image mode=RGB size=299x299 at 0x7A892EF43FE0>


In [7]:
x = np.array(img)
x.shape

(299, 299, 3)

In [8]:
tf.config.set_visible_devices([], 'GPU')

2025-11-30 16:34:35.442337: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [9]:
model = Xception(weights='imagenet', input_shape=(299, 299, 3))

In [10]:
X = np.array([x])

In [11]:
X = preprocess_input(X)
X[0]

array([[[ 0.4039216 ,  0.3411765 , -0.2235294 ],
        [ 0.4039216 ,  0.3411765 , -0.2235294 ],
        [ 0.41960788,  0.35686278, -0.20784312],
        ...,
        [ 0.96862745,  0.9843137 ,  0.94509804],
        [ 0.96862745,  0.9843137 ,  0.94509804],
        [ 0.96862745,  0.99215686,  0.9372549 ]],

       [[ 0.47450984,  0.4039216 , -0.12156862],
        [ 0.4666667 ,  0.39607847, -0.12941176],
        [ 0.45882356,  0.38823533, -0.15294117],
        ...,
        [ 0.96862745,  0.9764706 ,  0.9372549 ],
        [ 0.96862745,  0.9764706 ,  0.9372549 ],
        [ 0.96862745,  0.9764706 ,  0.92941177]],

       [[ 0.56078434,  0.48235297, -0.00392157],
        [ 0.5686275 ,  0.4901961 ,  0.00392163],
        [ 0.5686275 ,  0.49803925, -0.01176471],
        ...,
        [ 0.9607843 ,  0.96862745,  0.92156863],
        [ 0.9607843 ,  0.96862745,  0.92156863],
        [ 0.9607843 ,  0.96862745,  0.92156863]],

       ...,

       [[ 0.2941177 ,  0.18431377, -0.40392154],
        [ 0

In [12]:
pred = model.predict(X)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [13]:
decode_predictions(pred)

[[('n03595614', 'jersey', np.float32(0.68196297)),
  ('n02916936', 'bulletproof_vest', np.float32(0.03814005)),
  ('n04370456', 'sweatshirt', np.float32(0.034324754)),
  ('n03710637', 'maillot', np.float32(0.011354217)),
  ('n04525038', 'velvet', np.float32(0.0018453607))]]

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [15]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_ds = train_gen.flow_from_directory('./clothing-dataset-small/train/', target_size=(150, 150), batch_size=32)

Found 3073 images belonging to 10 classes.


In [16]:
train_ds.class_indices

{'dress': 0,
 'hat': 1,
 'longsleeve': 2,
 'outwear': 3,
 'pants': 4,
 'shirt': 5,
 'shoes': 6,
 'shorts': 7,
 'skirt': 8,
 't-shirt': 9}

In [17]:
X, y = next(train_ds)

In [18]:
X.shape

(32, 150, 150, 3)

In [19]:
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_ds = val_gen.flow_from_directory('./clothing-dataset-small/validation/', target_size=(150, 150), batch_size=32, shuffle=False)

Found 341 images belonging to 10 classes.


In [20]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))

#4 dimension
base = base_model(inputs, training=False)

#turns into 1 dimension
vectors = keras.layers.GlobalAveragePooling2D()(base)

outputs = keras.layers.Dense(10)(vectors)

model = keras.Model(inputs, outputs)

In [21]:
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

loss = keras.losses.CategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [22]:
#history = model.fit(train_ds, epochs=3, validation_data=val_ds)

In [23]:
#plt.plot(history.history['accuracy'], label='Train')
#plt.plot(history.history['val_accuracy'], label='Val')
#plt.xticks(np.arange(10))
#plt.legend()

In [32]:
def make_model(learning_rate=0.01):
    base_model = Xception(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    base_model.trainable = False

    ################################################################
    inputs = keras.Input(shape=(150, 150, 3))
    
    #4 dimension
    base = base_model(inputs, training=False)
    
    #turns into 1 dimension
    vectors = keras.layers.GlobalAveragePooling2D()(base)
    
    outputs = keras.layers.Dense(10)(vectors)
    
    model = keras.Model(inputs, outputs)

    ################################################################
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)
    
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

    return model

In [25]:
"""
scores = {}

for lr in [0.001, 0.01, 0.1]:
    print(lr)

    model = make_model(learning_rate=lr)
    history = model.fit(train_ds, epochs=3, validation_data=val_ds)
    scores[lr] = history.history
    
    print()
    print()
"""

'\nscores = {}\n\nfor lr in [0.001, 0.01, 0.1]:\n    print(lr)\n\n    model = make_model(learning_rate=lr)\n    history = model.fit(train_ds, epochs=3, validation_data=val_ds)\n    scores[lr] = history.history\n\n    print()\n    print()\n'

In [26]:
#scores

In [27]:
"""
for lr, hist in scores.items():
    print(lr)
    print(hist)
"""

'\nfor lr, hist in scores.items():\n    print(lr)\n    print(hist)\n'

In [28]:
"""
for lr, hist in scores.items(): 
    #plt.plot(hist['accuracy'], label=('train=%s' % lr))
    plt.plot(hist['val_accuracy'], label=('val=%s' % lr))
    plt.xticks(np.arange(10))
    plt.legend()
"""

"\nfor lr, hist in scores.items(): \n    #plt.plot(hist['accuracy'], label=('train=%s' % lr))\n    plt.plot(hist['val_accuracy'], label=('val=%s' % lr))\n    plt.xticks(np.arange(10))\n    plt.legend()\n"

In [36]:
model.save_weights("model_v1.weights.h5")

In [37]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_v1_{epoch:02d}_{val_accuracy:.3f}.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [ ]:
learning_rate = 0.001

model = make_model(learning_rate=learning_rate)
history = model.fit(train_ds, epochs=5, validation_data=val_ds)
callbacks=[checkpoint]

Epoch 1/5
81/97 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.4936 - loss: 1.5041